<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.30.2
    Uninstalling protobuf-6.30.2:
      Successfully uninstalled protobuf-6.30.2


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-14 09:29:56
-------------------
qualified stocks: 94
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  4.03 L
Current:  1.27 C
-------------------
Today PnL: 66.21 K (0.52%)
Current PnL: -18.23 L (-12.93%)
CY Booked + Current PnL: -15.36 L (-10.89%)
-------------------
Total profit:  3.93 L
Total loss:  -22.16 L
-------------------
Total Booked + Current PnL: 11.62 L (10.19%)
Total Booked PnL: 29.85 L (26.17%)
Curr Year Booked PnL: 2.87 L (2.25%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.08 C
Est FTT PnL: 80.11 L (62.86%)
-------------------
Est LTT:  2.69 C
Est LTT PnL: 1.42 C (111.25%)
Deployed:  1.14 C
Current:  1.27 C
CAGR/XIRR %: 8.04%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

Symbol    AvgCost        FTT  Dev%_PE Conviction    RSP   Current  \
53         MRF  114777.60  150504.14    14.94       X-MC  88.83  140940.0   
56  PIDILITIND    2504.06    3576.00   -15.90       X-LC  51.06   93240.0   
69     SBILIFE    1496.49    1924.99    -1.16       H-LC  80.85  146840.0   
70        SBIN     760.30     863.00   -12.51       M-LC  58.51  213537.0   
25       DMART    3484.00    5112.00   -20.84       X-LC  11.70   97493.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
53      26162.0   9570.0        0.37         22.79   6.79  31.13    215.0   
56      18118.0  14042.0        0.24         24.12  15.06  42.81     87.0   
69      21135.0  14860.0        0.47         16.81  10.12  28.63    116.0   
70      12058.0  15161.0        0.51          5.98   7.10  13.51     91.0   
25      13877.0  25192.0       -0.05         16.60  25.84  46.73    123.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
53     2.73        1.14    37.29       AR      ATH      TYRES  
56     1.29        0.76    16.93      X40      BTT  CHEMICALS  
69     1.42        1.19    30.82       AR      ATH  INSURANCE  
70     0.80        1.73    16.99     XY25      NTT      BANKS  
25     0.55        0.79    19.34       AR      NTT       FMCG

In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
73   SIEMENS  4406.90  7969.85   -16.48       X-LC  63.83   75615.0   
65      SAIL   130.64   228.00    81.22       M-MC  57.45  205951.0   
75  SONACOMS   578.05  1006.00   -29.33       M-MC  84.04   91438.0   
82  TITAGARH  1097.23  1548.00   -23.37       H-SC  14.89  134732.0   
27   FINEORG  4771.46  7168.71   -21.40       H-SC  35.11   74973.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
73     -34557.0  123631.0        3.70        -31.37  163.50  80.85     10.0   
65     -19011.0  186674.0        3.10         -8.45   90.64  74.53    217.0   
75      -9721.0   84617.0        2.82         -9.61   92.54  74.03     21.0   
82     -57283.0  136174.0        2.78        -29.83  101.07  41.08     76.0   
27      -6142.0   46896.0        2.61         -7.57   62.55  50.24    128.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy     Category  
73    -0.28        0.61    11.37       AR      ATH   ELECTRICAL  
65    -0.10        1.67    19.58     XY24      BTT        STEEL  
75    -0.11        0.74    28.50       AR      BTT         AUTO  
82    -0.42        1.09    14.03     XY24      NTT  ENGINEERING  
27    -0.13        0.61    25.89     X40N      ATH    CHEMICALS

In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

Symbol   AvgCost       FTT  Dev%_PE Conviction     RSP   Current  \
90      VIPIND    488.80    718.00  -792.09       H-SC  100.00   65930.0   
20   COFFEEDAY     59.14     80.00   -61.77       L-SC   59.57   56621.0   
34      HONAUT  42646.15  58518.52   -27.93       X-MC   41.49  105555.0   
80  TATAMOTORS    776.02   1065.00   -53.87       M-LC   81.91  184692.0   
37   INDIAMART   2311.97   4911.36   -52.77       H-SC   69.15  102076.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
90     -29386.0   74079.0       -2.61        -30.83  112.36   46.89     51.0   
20     -56928.0   96980.0       -2.03        -50.14  171.28   35.27    121.0   
34     -22383.0   70004.0       -1.76        -17.50   66.32   37.22     44.0   
80     -20953.0   97536.0       -1.52        -10.19   52.81   37.24      1.0   
37        349.0  114029.0       -1.30          0.34  111.71  112.43      NaN   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
90    -0.40        0.53    30.04     X40N      NTT        MISC  
20    -0.59        0.46    38.58       XR      NTT      HOTELS  
34    -0.32        0.86     7.76     X40N      ATH  ELECTRICAL  
80    -0.21        1.50    20.22     XY24      NTT        AUTO  
37     0.00        0.83    20.47       AR      ATH        MISC

In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
80  TATAMOTORS   776.02  1065.00   -53.87       M-LC  81.91  184692.0   
2          ABB  5319.37  8953.12   -37.83       H-LC  74.47  142212.0   
24         DLF   683.35  1031.70   -25.86       H-LC  44.68  122904.0   
10         AWL   342.88   485.00   -68.72       H-MC   7.45  169354.0   
92   WHIRLPOOL  1151.42  2270.00   -35.67       M-SC  92.55  161938.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
80     -20953.0   97536.0       -1.52        -10.19  52.81  37.24      1.0   
2        9228.0   81615.0        0.92          6.94  57.39  68.31      2.0   
24        -99.0   62804.0        0.73         -0.08  51.10  50.98      3.0   
10     -48375.0  138616.0        1.37        -22.22  81.85  41.45      5.0   
92      15708.0  126360.0        0.72         10.74  78.03  97.15      7.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
80    -0.21        1.50    20.22     XY24      NTT        AUTO  
2      0.11        1.15    15.96       AR      ATH  ELECTRICAL  
24    -0.00        1.00    12.29       AR      ATH      REALTY  
10    -0.35        1.37    13.13     XY24      NTT        FMCG  
92     0.12        1.31    38.34      X40      NTT    DURABLES

In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

Symbol    AvgCost        FTT  Dev%_PE Conviction    RSP   Current  \
53         MRF  114777.60  150504.14    14.94       X-MC  88.83  140940.0   
52     MEDANTA    1022.60    1486.00     6.60       X-MC  12.77  187978.0   
56  PIDILITIND    2504.06    3576.00   -15.90       X-LC  51.06   93240.0   
57     POLYCAB    5005.92    7541.35     7.45       H-MC  88.83  168210.0   
4   ADANIPORTS    1103.69    1583.00    -2.58       M-LC  86.17  188361.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
53      26162.0   9570.0        0.37         22.79   6.79  31.13    215.0   
52      32543.0  37896.0       -0.39         20.94  20.16  45.32    198.0   
56      18118.0  14042.0        0.24         24.12  15.06  42.81     87.0   
57      28044.0  42944.0        0.25         20.01  25.53  50.65     54.0   
4       37155.0  28518.0        0.42         24.57  15.14  43.43     37.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
53     2.73        1.14    37.29       AR      ATH       TYRES  
52     0.86        1.52    27.90     XY24      NTT  HEALTHCARE  
56     1.29        0.76    16.93      X40      BTT   CHEMICALS  
57     0.65        1.36    28.38     X40N      ATH      CABLES  
4      1.30        1.53    30.56     XY24      NTT        MISC

In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
57     POLYCAB  5005.92  7541.35     7.45       H-MC  88.83  168210.0   
6     ANGELONE  2328.67  3033.00     0.41       H-SC  73.40  164028.0   
4   ADANIPORTS  1103.69  1583.00    -2.58       M-LC  86.17  188361.0   
93       WIPRO   243.46   420.00    -8.35       M-LC  50.00  156835.0   
70        SBIN   760.30   863.00   -12.51       M-LC  58.51  213537.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
57      28044.0   42944.0        0.25         20.01  25.53  50.65     54.0   
6       12664.0   33117.0        1.44          8.37  20.19  30.25    131.0   
4       37155.0   28518.0        0.42         24.57  15.14  43.43     37.0   
93       5890.0  103558.0        0.55          3.90  66.03  72.51     58.0   
70      12058.0   15161.0        0.51          5.98   7.10  13.51     91.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
57     0.65        1.36    28.38     X40N      ATH   CABLES  
6      0.38        1.33    29.82     X40N      NTT  FINANCE  
4      1.30        1.53    30.56     XY24      NTT     MISC  
93     0.06        1.27    18.08       XR      NTT       IT  
70     0.80        1.73    16.99     XY25      NTT    BANKS

In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
93      WIPRO   243.46   420.00    -8.35       M-LC  50.00  156835.0   
12  BANKINDIA   113.49   190.00   -33.09       M-MC  45.74  130415.0   
37  INDIAMART  2311.97  4911.36   -52.77       H-SC  69.15  102076.0   
18       CAMS  3643.00  5250.99    -9.56       H-SC   5.32  101780.0   
91     VOLTAS  1272.00  1929.20   -30.46       H-MC  15.96  138985.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
93       5890.0  103558.0        0.55          3.90   66.03   72.51     58.0   
12       1263.0   85800.0        1.81          0.98   65.79   67.42     71.0   
37        349.0  114029.0       -1.30          0.34  111.71  112.43      NaN   
18       -224.0   45251.0        0.24         -0.22   44.46   44.14     47.0   
91       -935.0   73231.0        0.46         -0.67   52.69   51.67     19.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
93     0.06        1.27    18.08       XR      NTT       IT  
12     0.01        1.06    26.84       XR      NTT    BANKS  
37     0.00        0.83    20.47       AR      ATH     MISC  
18    -0.00        0.82    15.99       XR      ATH     MISC  
91    -0.01        1.13     4.98       AR      ATH       AC

In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

Symbol    AvgCost        FTT  Dev%_PE Conviction    RSP   Current  \
85  UJJIVANSFB      52.77      60.00    36.52       M-SC  77.66  118692.0   
12   BANKINDIA     113.49     190.00   -33.09       M-MC  45.74  130415.0   
53         MRF  114777.60  150504.14    14.94       X-MC  88.83  140940.0   
69     SBILIFE    1496.49    1924.99    -1.16       H-LC  80.85  146840.0   
86   UNIONBANK     123.87     163.00   -14.11       M-LC  72.34  151517.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
85     -23787.0  43311.0       -0.02        -16.70  36.49  13.70    219.0   
12       1263.0  85800.0        1.81          0.98  65.79  67.42     71.0   
53      26162.0   9570.0        0.37         22.79   6.79  31.13    215.0   
69      21135.0  14860.0        0.47         16.81  10.12  28.63    116.0   
86      10677.0  33819.0        1.90          7.58  22.32  31.59     83.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
85    -0.55        0.96    40.09     X40N      NTT      BANKS  
12     0.01        1.06    26.84       XR      NTT      BANKS  
53     2.73        1.14    37.29       AR      ATH      TYRES  
69     1.42        1.19    30.82       AR      ATH  INSURANCE  
86     0.32        1.23    31.61     XY24      NTT      BANKS

In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction     RSP   Current  \
90     VIPIND   488.80   718.00  -792.09       H-SC  100.00   65930.0   
58      QUESS   711.66   986.00      NaN        NaN   98.94   80482.0   
79  TATAELXSI  7332.28  9000.00   -20.01       X-MC   97.87   84056.0   
1    AARTIIND   487.04   919.00    36.75       M-SC   96.81   78457.0   
78      TANLA   917.30  1963.11   -39.81       H-SC   95.74  147139.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
90     -29386.0   74079.0       -2.61        -30.83  112.36   46.89     51.0   
58     -75372.0  135451.0        1.13        -48.36  168.30   38.55      NaN   
79     -18596.0   41944.0        0.30        -18.12   49.90   22.74     33.0   
1       -2879.0   75021.0        0.66         -3.54   95.62   88.69    118.0   
78     -92276.0  365228.0        1.15        -38.54  248.22  114.01     29.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
90    -0.40        0.53    30.04     X40N      NTT       MISC  
58    -0.56        0.65    23.24     XY24      NTT       MISC  
79    -0.44        0.68    26.35     X40N      NTT         IT  
1     -0.04        0.64    32.30       XR      NTT  CHEMICALS  
78    -0.25        1.19    34.95       AR      ATH         IT

In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

Top_N  Sum_Alloc%
0     10       19.30
1     25       40.96
2     50       69.51

In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    34.78
X40     14.46
X40N    14.01
AR      13.88
XR      10.30
XY25    10.23
X200     1.21
SR       1.10
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    22.31
X-LC    18.27
M-SC    16.60
M-LC     8.42
H-MC     7.82
X-MC     6.74
H-LC     5.68
M-MC     5.55
X-SC     2.81
L-SC     2.26
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

CurrAlloc%  Current P/L%    FTT%
Category                                       
FMCG                11.78         -1.97   34.41
BANKS               11.09         -9.97   63.91
IT                   9.31        -20.41   87.58
FINANCE              7.46        -20.15   74.37
MISC                 7.18        -27.73   82.70
INSURANCE            6.06         -3.33   34.48
PAINTS               5.56        -20.03   55.55
ELECTRICAL           4.12         -9.34   65.06
HEALTHCARE           3.65         -3.44   36.84
REFINERIES           3.65          9.67    6.98
CHEMICALS            3.29        -16.44   96.67
AUTO                 3.19        -15.50   67.20
FOOTWEAR             2.41        -38.26   83.95
DURABLES             1.99         -0.38   78.34
TRAVEL               1.95        -32.00   89.13
AC                   1.93        -12.67   72.79
TEXTILES             1.68        -10.41   68.40
CERAMICS             1.68        -46.32  108.11
STEEL                1.67         -9.23   90.64
CABLES               1.36         16.67   25.53
JEWELLERY            1.27        -83.58  141.62
TYRES                1.14         18.56    6.79
CEMENT               1.11        -37.81  108.72
ENGINEERING          1.09        -42.52  101.07
PHARMA               1.09          2.08   46.04
REALTY               1.00         -0.08   51.10
ENTERTAINMENT        0.91        -45.22  118.24
POWER                0.89        -28.91   80.32
HOTELS               0.46       -100.54  171.28

In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Criteria                  
XY24      3059788.0     27
XR        1270052.0     15
AR        1242496.0     15
X40N      1203397.0     17
X40        811450.0     12
SR         208900.0      2
XY25       146871.0      5
X200        67968.0      1

In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction                  
H-SC        2435018.0     22
M-SC        1955292.0     20
X-LC         726981.0     12
M-MC         583603.0      5
H-MC         431279.0      7
X-SC         361274.0      4
M-LC         341896.0      6
X-MC         323491.0      6
L-SC         318426.0      4
H-LC         225164.0      5

In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      930537.0      6
M-SC       XY24      771818.0      6
H-SC       X40N      557853.0      7
           AR        540024.0      3
M-SC       XR        430461.0      5
M-MC       XY24      413186.0      3
X-LC       X40       376960.0      5
M-SC       X40N      373559.0      5
H-SC       XR        305917.0      4
M-LC       XY24      223177.0      4
M-SC       X40       199819.0      2
H-LC       AR        191214.0      4
X-LC       AR        184293.0      3
H-MC       XY24      183068.0      2
L-SC       XR        164619.0      2
X-MC       XY24      141123.0      2
X-SC       SR        123386.0      1
           XR        120795.0      1
           X40N      117093.0      2
X-MC       X40N      111948.0      2
M-LC       XR        103558.0      1
H-SC       X40       100687.0      2
X-LC       XY25       97760.0      3
M-SC       AR         94121.0      1
L-SC       XY24       88381.0      1
M-MC       XR         85800.0      1
M-SC       SR         85514.0      1
M-MC       AR         84617.0      1
H-MC       AR         73231.0      1
           X40        73134.0      2
X-LC       X200       67968.0      1
L-SC       AR         65426.0      1
X-MC       X40        60850.0      1
H-MC       XR         58902.0      1
           X40N       42944.0      1
H-LC       XY25       33950.0      1
M-LC       XY25       15161.0      1
X-MC       AR          9570.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 45.0 seconds
